In [2]:
# Import the dependencies.
import pandas as pd
import numpy as np
from citipy import citipy
from datetime import datetime
import gmaps
import requests

# Import the API keys.
from config import weather_api_key
from config import g_key

In [3]:
df = pd.read_csv("Resources/City_Data_Challenge.csv")
df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain in Last 3 Hours (inches),Snow in Last 3 Hours (inches),Weather Description
0,0,Carnarvon,ZA,2019-10-22 06:22:58,-30.97,22.13,52.43,47,0,10.33,0.0,0.0,clear sky
1,1,Isla Mujeres,MX,2019-10-22 06:25:23,21.23,-86.73,82.40,83,5,9.17,0.0,0.0,clear sky
2,2,Arraial Do Cabo,BR,2019-10-22 06:22:39,-22.97,-42.02,70.20,74,76,11.52,0.0,0.0,broken clouds
3,3,Vestmanna,FO,2019-10-22 06:25:24,62.16,-7.17,41.00,75,83,10.29,0.0,0.0,light rain
4,4,Eltham,GB,2019-10-22 06:25:24,51.45,0.05,48.20,93,24,4.70,0.0,0.0,few clouds


In [40]:
# Ask the customer to add a minimum and maximum temperature value. 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [41]:
# Prompt the customer to answer if he or she would like it to be raining/snowing or not.
raining = input("Do you want it to be raining? (yes/no) ")
snowing = input("Do you want it to be snowing? (yes/no) ")

Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [42]:
# Filter the dataset to find the cities that fit the criteria.
preferred_temp_df = df.loc[(df["Max Temp"] <= max_temp) & \
                             (df["Max Temp"] >= min_temp)]

In [43]:
if raining == "no":
    preferred_rain_df = preferred_temp_df.loc[preferred_temp_df["Rain in Last 3 Hours (inches)"] == 0]
else:
    preferred_rain_df = preferred_temp_df.loc[preferred_temp_df["Rain in Last 3 Hours (inches)"] != 0]

In [44]:
if snowing == "no":
    preferred_cities_df = preferred_rain_df.loc[preferred_rain_df["Snow in Last 3 Hours (inches)"] == 0]
else:
    preferred_cities_df = preferred_rain_df.loc[preferred_rain_df["Snow in Last 3 Hours (inches)"] != 0]

In [45]:
preferred_cities_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain in Last 3 Hours (inches),Snow in Last 3 Hours (inches),Weather Description
1,1,Isla Mujeres,MX,2019-10-22 06:25:23,21.23,-86.73,82.40,83,5,9.17,0.0,0.0,clear sky
8,8,Shingu,JP,2019-10-22 06:25:25,33.72,135.99,75.20,57,40,16.11,0.0,0.0,scattered clouds
12,12,Kapaa,US,2019-10-22 06:22:34,22.08,-159.32,80.60,83,90,14.99,0.0,0.0,overcast clouds
19,19,Vaini,IN,2019-10-22 06:22:50,15.34,74.49,80.60,69,75,9.17,0.0,0.0,broken clouds
20,20,Ligayan,PH,2019-10-22 06:25:29,4.65,119.47,84.92,68,76,5.12,0.0,0.0,broken clouds


In [52]:
preferred_cities_df.count()

Unnamed: 0                       149
City                             149
Country                          148
Date                             149
Lat                              149
Lng                              149
Max Temp                         149
Humidity                         149
Cloudiness                       149
Wind Speed                       149
Rain in Last 3 Hours (inches)    149
Snow in Last 3 Hours (inches)    149
Weather Description              149
dtype: int64

In [58]:
preferred_cities_df[preferred_cities_df['Country'].isnull()]

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain in Last 3 Hours (inches),Snow in Last 3 Hours (inches),Weather Description
153,153,Rundu,NaN,2019-10-22 06:26:08,-17.91,19.77,80.6,14,77,4.7,0.0,0.0,broken clouds


In [59]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', "Rain in Last 3 Hours (inches)", "Snow in Last 3 Hours (inches)", "Weather Description"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain in Last 3 Hours (inches),Snow in Last 3 Hours (inches),Weather Description,Hotel Name
1,Isla Mujeres,MX,2019-10-22 06:25:23,21.23,-86.73,82.40,83,5,9.17,0.0,0.0,clear sky,
8,Shingu,JP,2019-10-22 06:25:25,33.72,135.99,75.20,57,40,16.11,0.0,0.0,scattered clouds,
12,Kapaa,US,2019-10-22 06:22:34,22.08,-159.32,80.60,83,90,14.99,0.0,0.0,overcast clouds,
19,Vaini,IN,2019-10-22 06:22:50,15.34,74.49,80.60,69,75,9.17,0.0,0.0,broken clouds,
20,Ligayan,PH,2019-10-22 06:25:29,4.65,119.47,84.92,68,76,5.12,0.0,0.0,broken clouds,
21,Riyadh,SA,2019-10-22 06:25:29,24.63,46.72,89.60,14,0,11.41,0.0,0.0,clear sky,
24,Airai,TL,2019-10-22 06:23:21,-8.93,125.41,85.12,25,2,9.35,0.0,0.0,clear sky,
30,Sao Filipe,CV,2019-10-22 06:22:48,14.90,-24.50,75.72,78,0,15.08,0.0,0.0,clear sky,
38,Atuona,PF,2019-10-22 06:25:00,-9.80,-139.03,78.60,77,20,12.82,0.0,0.0,few clouds,
40,Alice Springs,AU,2019-10-22 06:25:35,-23.70,133.88,89.60,6,0,13.87,0.0,0.0,clear sky,


In [60]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}

In [61]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to the params dictionary as values to 
    # the location key.
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and get the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [62]:
hotel_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain in Last 3 Hours (inches),Snow in Last 3 Hours (inches),Weather Description,Hotel Name
1,Isla Mujeres,MX,2019-10-22 06:25:23,21.23,-86.73,82.40,83,5,9.17,0.0,0.0,clear sky,Hotel La Joya Isla Mujeres
8,Shingu,JP,2019-10-22 06:25:25,33.72,135.99,75.20,57,40,16.11,0.0,0.0,scattered clouds,新宮ユーアイホテル
12,Kapaa,US,2019-10-22 06:22:34,22.08,-159.32,80.60,83,90,14.99,0.0,0.0,overcast clouds,Courtyard by Marriott Kaua'i at Coconut Beach
19,Vaini,IN,2019-10-22 06:22:50,15.34,74.49,80.60,69,75,9.17,0.0,0.0,broken clouds,Dandeli Lake County
20,Ligayan,PH,2019-10-22 06:25:29,4.65,119.47,84.92,68,76,5.12,0.0,0.0,broken clouds,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,Oeiras Do Para,BR,2019-10-22 06:28:13,-2.00,-49.85,75.43,84,0,3.94,0.0,0.0,clear sky,Araticu Soft Hotel
551,Yen Bai,VN,2019-10-22 06:28:14,21.70,104.87,85.51,61,74,3.36,0.0,0.0,broken clouds,AK MOTEL
554,Vaitape,PF,2019-10-22 06:28:15,-16.52,-151.75,81.18,69,0,5.35,0.0,0.0,clear sky,Conrad Bora Bora Nui
558,Norsup,VU,2019-10-22 06:28:16,-16.07,167.38,82.40,74,75,8.05,0.0,0.0,broken clouds,Nabelchel Bungalows


In [64]:
hotel_df.to_csv("Resources/vacation.csv")

In [74]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

In [75]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd>Hotel La Joya Isla Mujeres</dd>\n<dt>City</dt><dd>Isla Mujeres</dd>\n<dt>Country</dt><dd>MX</dd>\n<dt>Current Weather</dt><dd>clear sky and 82.4 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>新宮ユーアイホテル</dd>\n<dt>City</dt><dd>Shingu</dd>\n<dt>Country</dt><dd>JP</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 75.2 °F</dd>\n</dl>\n',
 "\n<dl>\n<dt>Hotel Name</dt><dd>Courtyard by Marriott Kaua'i at Coconut Beach</dd>\n<dt>City</dt><dd>Kapaa</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 80.6 °F</dd>\n</dl>\n",
 '\n<dl>\n<dt>Hotel Name</dt><dd>Dandeli Lake County</dd>\n<dt>City</dt><dd>Vaini</dd>\n<dt>Country</dt><dd>IN</dd>\n<dt>Current Weather</dt><dd>broken clouds and 80.6 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Ligayan</dd>\n<dt>Country</dt><dd>PH</dd>\n<dt>Current Weather</dt><dd>broken clouds and 84.92 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Al Salam Hotel</

In [77]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, 
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))